In [59]:
import numpy as np
import os
import cv2 as cv
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model


ImportError: Traceback (most recent call last):
  File "C:\Users\Machine Learning\AppData\Local\conda\conda\envs\machinelearning\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "C:\Users\Machine Learning\AppData\Local\conda\conda\envs\machinelearning\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Machine Learning\AppData\Local\conda\conda\envs\machinelearning\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Machine Learning\AppData\Local\conda\conda\envs\machinelearning\lib\site-packages\tensorflow

In [38]:
### Hyperparamters
IMGH=200
IMGW=200
datasetFolder=r'dataset'

In [40]:
## Importing data
imgData=[]
labels=[]
for folder in os.listdir(datasetFolder):
    for filename in os.listdir(os.path.join(datasetFolder,folder)):
        imagePath=os.path.join(datasetFolder,folder,filename)
        image=cv.imread(imagePath,cv.COLOR_BGR2RGB)
        image=cv.resize(image, (IMGH, IMGW),interpolation = cv.INTER_AREA)
        image=np.array(image)
        image=image.astype('float32')
        image=image/255
        imgData.append(image)
        labels.append(folder)

In [57]:
print(imgData[0].shape)
print(labels[0:10])

(200, 200, 3)
['female', 'female', 'female', 'female', 'female', 'female', 'female', 'female', 'female', 'female']


In [48]:
## Train test split
Xtrain,yTrain,Xtest,yTest=train_test_split(imgData,labels,random_state=0,test_size=0.33)

In [52]:
target_dict={'male':0,'female':1}

In [ ]:
### CNN
model=keras.Sequential[
    InputLayer(input_shape=(IMGH,IMGW,3)),
    Conv2D(filters=32,kernal_size=3,strides=(2, 2), activation='relu'),
    Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    Flatten(),
    Dense(2)
]